# Recuperation et analyse de donnée Twitter

Ici nous allons utiliser la bibliotèque tweepy pour effectuer des recherche et recuperer un flux twitter et ensuite nous verrons la methode pour installer tout ceci sur un raspberrypi pi pour automatiser la recuperation.

En premier lieu il nous faut des clé d'identification.
Ces clés sont obtenables en s'incrivant sur twitter et créant une app twitter ( https://developer.twitter.com/en/apps ) et sont necaissaire pour l'utilisation de la bibliothèque.

In [1]:
consumer_key= 'ErI2gyTmnaE3YGeAz4DWc62nX'
consumer_secret= 'OzSy0s4ma6CDVxwZgLisFdqmguDLQrsYaxqpCEqE31He8UgupR'
access_token= '1229729717806358529-iXdWrMVhtYT8NuPQyT8ldOTr1k3QRz'
access_token_secret= 'hDHXzyuAJFgvWAodA74KsJYtB9cUPKTP5CQ7snCWDX3Hh'

## Imports et Fonctions

In [2]:
import matplotlib.pyplot as plt
import re, string, random
import numpy as np
import pandas as pd
import time
import json
import csv
import pickle

import tweepy as tw
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier

from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization
from keras.optimizers import RMSprop

nltk.download('twitter_samples')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('punkt')

stop_words = stopwords.words('english')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Unzipping corpora/twitter_samples.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


### Mise en place du Classifieur

Les classifieurs choisi sont 4 clasiifieurs entrainés sur le dataset reduit avec le reseau de neuronne pré-entrainé du notebook reduction.Et ensuite on prend la classe la plus predite par ces classifieur.

In [3]:
def remove_noise(tweet_tokens, stop_words = ()): 

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens


def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

def make_set( tweet_tokens, label="", stop_words = () ) :
    cleaned_tokens_list = []
    for tokens in tweet_tokens:
        cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    tokens_for_model = get_tweets_for_model(cleaned_tokens_list)
    dataset = [(tweet_dict, label)for tweet_dict in tokens_for_model]
    return dataset

def to_numpy(dataset,vocab=None):

    V = [] #vacobulaire
    Data = [] #features
    C = [] #classe

    if vocab==None:

#vacobulaire     
      for tweet in dataset:
        for token in tweet[0].keys():
          V.append(token)
      V = np.unique(V)

      for tweet in dataset:
#features
        x = np.zeros(V.shape)
        for token in tweet[0].keys():
          x += (token == V)
        Data.append(x)
#classe
        if tweet[1] == 'Positive':
          C.append([1,0,0])
        elif tweet[1] == 'Negative':
          C.append([0,1,0])
        else :
          C.append([0,0,1])

    else:
      V = np.array(vocab)
      for tweet in dataset:
  #features
        x = np.zeros(V.shape)
        for token in tweet[0].keys():
          x += (token == V)
        Data.append(x)
  #classe
        if tweet[1] == 'Positive':
          C.append([1,0,0])
        elif tweet[1] == 'Negative':
          C.append([0,1,0])
        else :
          C.append([0,0,1])

    
    Xtrain = Data[:11000]
    Xtest = Data[11000:]
    Xtrain = (np.array(Xtrain)!=0) *1
    Xtest = (np.array(Xtest)!=0) *1

    Ytrain = np.array(C[:11000])
    Ytest = np.array(C[11000:])

    return Xtest,Xtrain,Ytest,Ytrain,V

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tweet_tokens = np.random.choice(twitter_samples.tokenized('tweets.20150430-223406.json'),size=len(negative_tweet_tokens),replace = False)

positive_dataset = make_set( positive_tweet_tokens, label="Positive", stop_words = stop_words )
negative_dataset = make_set( negative_tweet_tokens, label="Negative", stop_words = stop_words )
neutral_dataset = make_set( neutral_tweet_tokens, label="Neutral", stop_words = stop_words )

dataset = positive_dataset + negative_dataset + neutral_dataset
random.shuffle(dataset)

In [5]:
#reduction de la dimention
with open("vocab_reduc_part0.txt", "rb") as f:
    V = pickle.load(f)

Xtest,Xtrain,Ytest,Ytrain,V = to_numpy(dataset,vocab=V)
part0 = load_model('reduc_part0.h5')

Xtest_red,Xtrain_red = np.array(part0(Xtest)),np.array(part0(Xtrain))
Ytest,Ytrain = (Ytest[:,0]==1)*1 + (Ytest[:,1]==1)*2 + (Ytest[:,2]==1)*3, (Ytrain[:,0]==1)*1+(Ytrain[:,1]==1)*2+(Ytrain[:,2]==1)*3

In [6]:
#entrainement des classifieurs
CL = [GaussianNB(),
      DecisionTreeClassifier(max_depth=4),
      SVC(),
      MLPClassifier(hidden_layer_sizes=(10,10,10,10,10),max_iter=5000), ]

NCL = ['GaussianNB',
       'DecisionTreeClassifier profondeur 4',
       'SVC kernel rbf',
       'MLPClassifier 5 couches cachées',]

for clf in CL:
    clf_sentiment = clf.fit(Xtrain_red,Ytrain)

In [7]:
#mise en place de la methode ensembliste
eclf = VotingClassifier(estimators=[ (NCL[i],CL[i]) for i in range(len(NCL)) ],
                        voting='hard')

eclf = eclf.fit(Xtrain_red,Ytrain)

y_pred = eclf.predict(Xtest_red)

print(metrics.classification_report(Ytest,y_pred))


              precision    recall  f1-score   support

           1       0.99      1.00      0.99      1334
           2       1.00      0.99      0.99      1295
           3       0.99      0.99      0.99      1371

    accuracy                           0.99      4000
   macro avg       0.99      0.99      0.99      4000
weighted avg       0.99      0.99      0.99      4000



## Recherche twitter

Nous allons voir quelques exemple de recherche de tweet sur quelques sujets. Pour plus de detail sur les filtre de recherche utilisable https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators.

Il est bon de savoir aussi qu'il y à plusieurs contraintes, sur cet outil de recherche:
 
* La limite en nombre de requete est 180/15 minutes et on ne peu aller au-delà de 7 jour pour un sujet donné.
* Pour recuperer la timeline d'un user la limite est de 900/15 minutes,avec 200/par requette sans la limite de es 7 jour. Mais l'api tweeter ne semble pas avoir d'outils pour aller plus vieux que les 200 plus recents.



In [9]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=False)

In [27]:
N = 20
search_words = "New York  -filter:retweets"
date_since = "2020-01-01"
R = tw.Cursor(api.search,q=search_words,lang="en",since=date_since,result_type = "recent" ).items(N)

infos = []

V = np.array(V)
for tweet in R :

  Txt = remove_noise(word_tokenize(tweet.text))
  txt_vec = np.zeros(np.shape(V))
  for token in Txt:
    txt_vec += token == V
  txt_vec = txt_vec!=0

  txt_vec_red = np.array(part0(txt_vec.reshape(1,-1)))

  s_tweet = eclf.predict(txt_vec_red) 
  if s_tweet==1 :
    s_tweet = "positif"
  elif s_tweet==2 :
    s_tweet = "negatif"
  else:
    s_tweet = "neutre"
    
  infos.append([tweet.created_at ,
              tweet.author.screen_name,
              tweet.author.location,
              tweet.author.followers_count,
              tweet.text,
              s_tweet,
              ])

Mesure = pd.DataFrame(data=infos,columns=["time","screen_name", "location","followers_count","text","sentiment_tweet"],)

In [28]:
Mesure

,time,screen_name,location,followers_count,text,sentiment_tweet
0,2020-08-24 08:58:01,Juice_Berry80,Perth Australia,89,@gfollis @tonilontis @MagdaSzubanski Controls ...,neutre
1,2020-08-24 08:58:00,livetennis,Tennis!,519948,Novak Djokovic played his last ATP match on 29...,negatif
2,2020-08-24 08:57:53,devdelevingne,"Long Island, NY",4378,i’m literally just a dummy for new york 🤡,neutre
3,2020-08-24 08:57:52,newmystic,"Fort Worth, TX",64,"@ANNELAMOTT oh, those real estate ones like Mi...",neutre
4,2020-08-24 08:57:44,511NY,New York State,2592,Cleared: Incident on #CoastLine at New York Pe...,positif
5,2020-08-24 08:57:43,511NY,New York State,2592,Cleared: Construction on #NortheastLine Both d...,negatif
6,2020-08-24 08:57:43,511NYC,New York City,2484,Cleared: Construction on #NortheastLine Both d...,negatif
7,2020-08-24 08:57:43,511NY,New York State,2592,Cleared: Construction on #MidtownDirect Both d...,negatif
8,2020-08-24 08:57:43,511NYC,New York City,2484,Cleared: Construction on #MidtownDirect Both d...,negatif
9,2020-08-24 08:57:43,511NY,New York State,2592,Cleared: Construction on #NorthLine Both direc...,negatif


In [36]:
N = 20
search_words = "from:realDonaldTrump -filter:retweets"
date_since = "2020-01-01"

R = tw.Cursor(api.search,q=search_words,lang="en",since=date_since,result_type = "recent" ).items(N)

infos = []

V = np.array(V)
for tweet in R :

  Txt = remove_noise(word_tokenize(tweet.text))
  txt_vec = np.zeros(np.shape(V))
  for token in Txt:
    txt_vec += token == V
  txt_vec = txt_vec!=0

  txt_vec_red = np.array(part0(txt_vec.reshape(1,-1)))

  s_tweet = eclf.predict(txt_vec_red) 
  if s_tweet==1 :
    s_tweet = "positif"
  elif s_tweet==2 :
    s_tweet = "negatif"
  else:
    s_tweet = "neutre"
    
  infos.append([tweet.created_at ,
              tweet.author.screen_name,
              tweet.author.location,
              tweet.author.followers_count,
              tweet.text,
              s_tweet,
              ])

Mesure = pd.DataFrame(data=infos,columns=["time","screen_name", "location","followers_count","text","sentiment_tweet"],)

In [37]:
Mesure

,time,screen_name,location,followers_count,text,sentiment_tweet
0,2020-08-24 02:00:38,realDonaldTrump,"Washington, DC",85555361,Continued... https://t.co/HNNyF3B5NM,neutre
1,2020-08-24 01:59:24,realDonaldTrump,"Washington, DC",85555361,Please watch highly respected Dr. Harvey Risch...,negatif
2,2020-08-24 01:23:25,realDonaldTrump,"Washington, DC",85555361,The greatest Election Fraud in our history is ...,neutre
3,2020-08-24 00:02:24,realDonaldTrump,"Washington, DC",85555361,"Watch Mark Levin, on air now. Great! @FoxNews",neutre
4,2020-08-23 18:45:19,realDonaldTrump,"Washington, DC",85555361,"Actually, I think I’m leading in the Polls! @...",neutre
5,2020-08-23 18:37:58,realDonaldTrump,"Washington, DC",85555361,Chris Wallace “forgot” to ask a very weak and ...,neutre
6,2020-08-23 18:29:03,realDonaldTrump,"Washington, DC",85555361,“Next we’ll turn to people of the Biden inner ...,neutre
7,2020-08-23 17:33:53,realDonaldTrump,"Washington, DC",85555361,Important White House News Conference at 5:30 ...,neutre
8,2020-08-23 13:48:46,realDonaldTrump,"Washington, DC",85555361,Happy Sunday! We want GOD! \nhttps://t.co/RsBk...,positif
9,2020-08-23 13:11:59,realDonaldTrump,"Washington, DC",85555361,Two Democrat Caucus Meetings removed “UNDER GO...,neutre


## streaming de twitter

Pour recuperer le flux twitter et l'nregistrer il va nous falloir plusieurs fonctions et quelques imports:

    from tweepy.streaming import StreamListener
    import time
    import json
    import csv
    import numpy as np
    ckey='ErI2gyTmnaE3YGeAz4DWc62nX'
    csecret='OzSy0s4ma6CDVxwZgLisFdqmguDLQrsYaxqpCEqE31He8UgupR'
    atoken='1229729717806358529-iXdWrMVhtYT8NuPQyT8ldOTr1k3QRz'
    asecret='hDHXzyuAJFgvWAodA74KsJYtB9cUPKTP5CQ7snCWDX3Hh'

Les tweets et leurs information associée arrivent en format Json mais si les colonnes sont vide rien ne l'indique et il faut alors parcourir l'ensemble des information pour recuperer l'ensemble des colonnes:

      def recup_keys(D):
          keys = list(D[0].keys())
          for i in range(1,len(D)):
              k=list(D[i].keys())
              for j in range(len(k)):
                  keys.append(k[j])

          keys = list(np.unique(keys))
          return keys

      def to_csv(D,name):
          keys = recup_keys(D)
          with open(name, 'a',encoding="utf-8") as f:  
              w = csv.DictWriter(f,fieldnames=keys)
              w.writeheader()

              for i in range(len(D)):
                  w.writerow(D[i])

Enfin voici le listener une sous-classe du StreamListener de tweepy ,avec 4 attributs suplementaires:
* D un liste de chaques tweets et informations associées 
* name qui servira lors de l'enregistrement.
* path le chemin d'accès pour l'enregistrement du csv, ne pas faire en chemin relatif cela évite de perdre du temps.
* size taille de la liste, ici 10 tweets et ses informations mais modifiable pour l'adapter à sa quantité de RAM ou à la frequence des coupure internet ou de courant.

La methode on_data filtre les retweet et lorsque la liste est remplie l'enregistre en format csv.
                  
      class listener(StreamListener):
          
          def __init__(self,name,path,size=10):
              self.D = []
              self.name = name
              self.path = path
              self.size = size

          def on_data(self, data):
              
              all_data = json.loads(data)
              tweet = all_data["text"]

              if tweet[0:2] != 'RT':
                  self.D.append(all_data)
                  #print('test')
                  if len(self.D) == self.size:
                      to_csv(self.D, self.path +'datatweet_' + str(self.name )+ str(time.time()) + '.csv')
                      self.D=[]

              return True

          def on_error(self, status):
              print (status)

Et enfin pour recuper les donnée , dans un fichier tweetstreaming.py mettre les fonction precedente et les imports et faire un second fichier,par exemple recherche.py, dans le même repertoire:

    import tweetstreaming as ts

    from tweepy import Stream
    from tweepy import OAuthHandler

    auth = OAuthHandler(ts.ckey, ts.csecret)
    auth.set_access_token(ts.atoken, ts.asecret)

    twitterStream = Stream(auth, ts.listener(name="nom"))
    twitterStream.filter(track=["vos","mots","clé"])

Pour finir vous pouvez le lancer en ligne de commande dans windows ou linux ou dans un environnement de développement.
Il aussi possible de le mettre en tache de fond, c'est ce que j'ai fait sur un raspberrypi. 

* Avoir un raspberrypi et installer la distibution linux raspbian(https://www.raspberrypi.org/documentation/installation/installing-images/) ou bien un ordinateur avec un os linux.
* Mettre dans un même fichier tweetstreaming.py et recherche.py.
* (facultatif)  Lancer un test avec un size petit et un mot clé très courant pour verifier si l'ensemble des fonction marche bien et la présence de toute les bibliothèque requise (le cas écheant les installer avec pip).
* Editons ensuite /etc/rc.local et avant la ligne 

      exit 0

  mettre:
      /usr/bin/python3 /home/pi/monprojet/recherche.py &

* Redémarrer, et le programme devrait etre lancé(Attention, un programme lancé de cette manière est un programme lancé en root, avec toutes les permissions en lecture comme en écriture).

Le nombre de tweets est dépendant des mots clés, il peut être normal d'avoir des resultat qu'au bout d'un certain temps.

Après ceci nous avons plusieurs fichiers du type datatweet_name_date.csv, alors pour les fusionner :

      import pandas as pd
      import os

      rep = r"path"

      Listecsv = os.listdir(rep)

      a = pd.read_csv(rep + Listecsv[0])

      for f in Listecsv[1:]:
          
          b = pd.read_csv(rep + f)
          a = pd.concat((a,b))
          
      print(sum(a['lang']=='en'))

      a.to_csv('sujetdataset.csv')

      #a = pd.read_csv('sujetdataset.csv')

il faudrat veiller à mettre l'ensemble des csv dans le même repertoire sans aucun autre fichier et en renseignant ce repertoire dans path, puis executer le code ci-dessus.

## Documentation Utilisé

-https://www.earthdatascience.org/courses/use-data-open-source-python/intro-to-apis/twitter-data-in-python/

-https://pythonprogramming.net/twitter-api-streaming-tweets-python-tutorial/

-https://developer.twitter.com/en/docs/tweets/rules-and-filtering/overview/standard-operators

-https://pythonprogramming.net/twitter-api-streaming-tweets-python-tutorial/

-https://developer.twitter.com/en/docs/twitter-api/v1/tweets/post-and-engage/overview

-documentation tweepy : http://docs.tweepy.org/en/latest/
